In [1]:
import csv
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

RANDOM_SEED = 42

2023-01-31 14:51:45.495771: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# dataset = Path('gestures_dataset_new.csv')
DATASET_CSV = Path('../gestures_dataset.csv')
DATASET_ROOT = Path('../ai_data/vgt-all')

UPLOADED_DATASET_CSV = Path('../uploaded_gestures_dataset.csv')
UPLOADED_GESTURES_ROOT = Path('../ai_data/vgt-uploaded')

In [3]:
def handedness_data_from_directory_names(dataset_location: Path):
    handedness_data = {}
    for gesture_folder in os.listdir(dataset_location):
        gesture_name, handedness_string = gesture_folder.split('_')
        handedness_data[gesture_name] = (handedness_string[0] == '1', handedness_string[1] == '1')
    return handedness_data

In [4]:
%load_ext autoreload
%autoreload 2
from sl_ai.gesture_classifier import GestureClassifier
from sl_ai.dataset import GestureDataset

gesture_dataset: GestureDataset = GestureDataset()
# gesture_dataset.scan_videos(dataset_location=DATASET_ROOT, handedness_data=handedness_data_from_directory_names(DATASET_ROOT))
# gesture_dataset.analyze_videos(csv_out_path=DATASET_CSV, overwrite=True)
gesture_dataset.load_from_csv(DATASET_CSV)


for user_folder in os.listdir(UPLOADED_GESTURES_ROOT):
    for gesture_folder in os.listdir(UPLOADED_GESTURES_ROOT / user_folder):
        csv_files = list(
            filter(
                lambda file: file.endswith(".csv"),
                os.listdir(
                    UPLOADED_GESTURES_ROOT / user_folder / gesture_folder
                ),
            )
        )
        if not csv_files:
            print(
                "Warning: Found a uploaded gesture folder without a dataset.csv file."
            )
            continue
        dataset_file = csv_files[0]
        print(
            f"Loading dataset from {UPLOADED_GESTURES_ROOT / user_folder / gesture_folder / dataset_file}"
        )
        *gestures_words, handedness_string = gesture_folder.split("_")
        gesture_name = "_".join(gestures_words)
        uploaded_gesture_dataset = GestureDataset(single_gesture=True)
        uploaded_gesture_dataset.scan_videos(
            dataset_location=UPLOADED_GESTURES_ROOT
            / user_folder
            / gesture_folder,
            handedness_data={
                gesture_name: (
                    handedness_string[0] == "1",
                    handedness_string[1] == "1",
                )
            },
        )
        uploaded_gesture_dataset.load_from_csv(
            csv_path=UPLOADED_GESTURES_ROOT
            / user_folder
            / gesture_folder
            / dataset_file
        )
        gesture_dataset.append_dataset(uploaded_gesture_dataset)

gesture_dataset.summary()

classifier: GestureClassifier = GestureClassifier(gesture_dataset=gesture_dataset)

belgië(0 -> 0)
bowlen(1 -> 1)
computer(2 -> 2)
duidelijk(3 -> 3)
fietsen(4 -> 4)
gebaar(5 -> 5)
geit(6 -> 6)
hallo(7 -> 7)
kameel(8 -> 8)
muis(9 -> 9)
tennis(10 -> 10)
verenigde staten(11 -> 11)
Something went wrong while processing usa - 2023-01-04_16-18-24.mp4: list index out of range


NotADirectoryError: [Errno 20] Not a directory: '../ai_data/vgt-uploaded/.DS_Store'

In [ ]:
gesture_dataset.x_data.shape

In [ ]:
gesture_dataset.y_data.shape

In [ ]:
len(gesture_dataset)

In [ ]:
classifier.train(train_size=.5)
classifier.summary()

In [ ]:
classifier.visualize_accuracy()

In [ ]:
classifier.visualize_loss()

In [ ]:
classifier.confusion_matrix()

In [ ]:
# classifier.save_model("../model.h5")

In [ ]:
exit()

------------

# Test predictions

In [ ]:
from sl_ai.dataset import detect_hands_task, GestureData

video_path = Path('../ai_data/vgt-all/duidelijk_11/duidelijk - 2023-01-04_16-41-21.mp4')
gesture_data = GestureData(name="hello", left_hand=False, right_hand=True)
video_name, (frame_width, frame_height), left_landmarks, right_landmarks = detect_hands_task(gesture_data, video_path)


In [ ]:
from sl_ai.dataset import preprocess_landmarks
from copy import deepcopy
left_landmarks_copy = deepcopy(left_landmarks)
right_landmarks_copy = deepcopy(right_landmarks)
preprocess_landmarks(left_landmarks_copy, right_landmarks_copy, None, None)

In [ ]:
from sl_ai.dataset import pre_process_point_history_center

for i, landmarks in left_landmarks_copy.items():
    left_landmarks_copy[i] = pre_process_point_history_center(None, None, landmarks)
for i, landmarks in right_landmarks_copy.items():
    right_landmarks_copy[i] = pre_process_point_history_center(None, None, landmarks)

In [ ]:
result = classifier.predict(left_landmarks_copy, right_landmarks_copy)
classes_x = np.argmax(result, axis=1)
print(classes_x)
prediction_percents = (result*100)
frame = pd.DataFrame(prediction_percents.astype(np.int32))
print(frame)

In [ ]:
result

In [ ]:
# l = np.array(left_landmarks_copy[12], dtype='float32')
# r = np.array(right_landmarks_copy[12], dtype='float32')
#
# x_data = np.concatenate((l, r), axis=0)
# x_data = x_data.reshape((-1, x_data.shape[0]))
# results = classifier.model.predict(x_data, batch_size=1)
# classes_x = np.argmax(result, axis=1)
# print(classes_x)
# prediction_percents = (result*100)
# frame = pd.DataFrame(prediction_percents.astype(np.int32))
# print(frame)

In [ ]:
# classifier.train(train_size=.5)
# classifier.summary()

In [ ]:
# classifier.visualize_accuracy()

In [ ]:
# classifier.visualize_loss()

In [ ]:
# classifier.confusion_matrix()

# Test predictions

In [ ]:
# from sl_ai.dataset import detect_hands_task, GestureData
#
# video_path = Path('../ai_data/vgt-all/duidelijk_11/duidelijk - 2023-01-04_16-41-21.mp4')
# gesture_data = GestureData(name="hello", left_hand=False, right_hand=True)
# video_name, (frame_width, frame_height), left_landmarks, right_landmarks = detect_hands_task(gesture_data, video_path)


In [ ]:
# from sl_ai.dataset import preprocess_landmarks
# from copy import deepcopy
# left_landmarks_copy = deepcopy(left_landmarks)
# right_landmarks_copy = deepcopy(right_landmarks)
# preprocess_landmarks(left_landmarks_copy, right_landmarks_copy, None, None)

In [ ]:
# from sl_ai.dataset import pre_process_point_history_center
#
# for i, landmarks in left_landmarks_copy.items():
#     left_landmarks_copy[i] = pre_process_point_history_center(None, None, landmarks)
# for i, landmarks in right_landmarks_copy.items():
#     right_landmarks_copy[i] = pre_process_point_history_center(None, None, landmarks)

In [ ]:
# result = classifier.predict(left_landmarks_copy, right_landmarks_copy)
# classes_x = np.argmax(result, axis=1)
# print(classes_x)
# prediction_percents = (result*100)
# frame = pd.DataFrame(prediction_percents.astype(np.int32))
# print(frame)

In [ ]:
# result

In [ ]:
# l = np.array(left_landmarks_copy[12], dtype='float32')
# r = np.array(right_landmarks_copy[12], dtype='float32')
#
# x_data = np.concatenate((l, r), axis=0)
# x_data = x_data.reshape((-1, x_data.shape[0]))
# results = classifier.model.predict(x_data, batch_size=1)
# classes_x = np.argmax(result, axis=1)
# print(classes_x)
# prediction_percents = (result*100)
# frame = pd.DataFrame(prediction_percents.astype(np.int32))
# print(frame)